<img src = "https://images2.imgbox.com/60/09/VFwl5LOq_o.jpg" width="400">

# 4. Beyond window functions
---

In this last chapter, you'll learn some techniques and functions that are useful when used together with window functions.

In [1]:
%load_ext sql

In [2]:
%sql sqlite:///data/summer.db

'Connected: @data/summer.db'

## A basic pivot
---

You have the following table of Pole Vault gold medalist countries by gender in 2008 and 2012.

`| Gender | Year | Country |
 |--------|------|---------|
 | Men    | 2008 | AUS     |
 | Men    | 2012 | FRA     |
 | Women  | 2008 | RUS     |
 | Women  | 2012 | USA     |`

Pivot it by Year to get the following reshaped, cleaner table.

`| Gender | 2008 | 2012 |
 |--------|------|------|
 | Men    | AUS  | FRA  |
 | Women  | RUS  | USA  |`

### Instructions

Create the correct extension.

Fill in the column names of the pivoted table.

CREATE EXTENSION IF NOT EXISTS tablefunc;

SELECT * FROM CROSSTAB($$
  SELECT
    gender, year, country
  FROM Summer_Medals
  WHERE
    Year IN (2008, 2012)
    AND medal = 'Gold'
    AND event = 'Pole Vault'
  ORDER By gender ASC, year ASC;
$$) AS ct (gender VARCHAR,
           "2008" VARCHAR,
           "2012" VARCHAR)

ORDER BY Gender ASC

## Pivoting with ranking
---

You want to produce an easy scannable table of the rankings of the three most populous EU countries by how many gold medals they've earned in the 2004 through 2012 Olympic games. The table needs to be in this format:

`| Country | 2004 | 2008 | 2012 |
 |---------|------|------|------|
 | FRA     | ...  | ...  | ...  |
 | GBR     | ...  | ...  | ...  |
 | GER     | ...  | ...  | ...  |`

You'll need to count the gold medals each country has earned, produce the ranks of each country by medals earned, then pivot the table to this shape.

### Instructions

Count the gold medals that France (`FRA`), the UK (`GBR`), and Germany (`GER`) have earned per country and year.

In [5]:
%%sql

SELECT country,
       year,
       COUNT(*) AS Awards
FROM   Summer_Medals
WHERE  country IN ( 'FRA', 'GBR', 'GER' )
       AND year IN ( 2004, 2008, 2012 )
       AND medal = 'Gold'
GROUP  BY country,
          year
ORDER  BY country ASC,
          year ASC 

 * sqlite:///data/summer.db
Done.


Country,Year,Awards
FRA,2004,21
FRA,2008,25
FRA,2012,30
GBR,2004,17
GBR,2008,31
GBR,2012,48
GER,2004,41
GER,2008,42
GER,2012,45


Select the country and year columns, then rank the three countries by how many gold medals they earned per year.

`WITH Country_Awards 
     AS (SELECT country,
                year,
                COUNT(*) AS Awards
         FROM   Summer_Medals
         WHERE  country IN ( 'FRA', 'GBR', 'GER' )
                AND year IN ( 2004, 2008, 2012 )
                AND medal = 'Gold'
         GROUP  BY country,
                   year)
SELECT country,
       year,
       RANK() OVER (PARTITION BY year 
                    ORDER BY Awards DESC) :: INTEGER AS rank
FROM   Country_Awards 
ORDER  BY country ASC,
          year ASC`

Pivot the query's results by `Year` by filling in the new table's correct column names.

In [ ]:
CREATE EXTENSION IF NOT EXISTS tablefunc;

SELECT * FROM CROSSTAB($$
  WITH Country_Awards AS (
    SELECT country,
           year,
           COUNT(*) AS Awards
    FROM Summer_Medals
    WHERE
      country IN ('FRA', 'GBR', 'GER')
      AND year IN (2004, 2008, 2012)
      AND medal = 'Gold'
    GROUP BY country, year)

  SELECT country,
         year,
         RANK() OVER (PARTITION BY year 
                      ORDER BY Awards DESC) :: INTEGER AS rank
  FROM Country_Awards
  ORDER BY country ASC, year ASC;
$$) AS ct (country  VARCHAR,
           "2004" INTEGER,
           "2008" INTEGER,
           "2012" INTEGER)

ORDER BY country ASC

## Country-level subtotals
---

You want to look at three Scandinavian countries' earned gold medals per country and gender in the year 2000. You're also interested in `Country`-level subtotals to get the total medals earned for each country, but `Gender`-level subtotals don't make much sense in this case, so disregard them.

### Instructions

Count the gold medals awarded per country and gender.

Generate `Country`-level gold award counts.

`SELECT country,
       gender,
       COUNT(*) AS Gold_Awards
FROM   Summer_Medals
WHERE  year = 2004
       AND medal = 'Gold'
       AND country IN ( 'DEN', 'NOR', 'SWE' )
GROUP  BY country,
          ROLLUP( gender )
ORDER  BY country ASC,
          gender ASC `

## All group-level subtotals
---

You want to break down all medals awarded to Russia in the 2012 Olympic games per gender and medal type. Since the medals all belong to one country, Russia, it makes sense to generate all possible subtotals (`Gender`- and `Medal`-level subtotals), as well as a grand total.

Generate a breakdown of the medals awarded to Russia per country and medal type, including all group-level subtotals and a grand total.

### Instructions

Count the medals awarded per gender and medal type.

Generate all possible group-level counts (per gender and medal type subtotals and the grand total).

`SELECT gender,
       medal,
       COUNT(*) AS Awards
FROM   Summer_Medals
WHERE  year = 2012
       AND country = 'RUS'
GROUP  BY CUBE( gender, medal )
ORDER  BY gender ASC,
          medal ASC`

## Cleaning up results
---

Returning to the breakdown of Scandinavian awards you previously made, you want to clean up the results by replacing the `null`s with meaningful text.

### Instructions

Turn the `null`s in the `Country` column to `All countries`, and the `null`s in the `Gender` column to `All genders`.

`SELECT COALESCE(country, 'All countries') AS Country,
       COALESCE(gender, 'All genders')    AS Gender,
       COUNT(*)                           AS Awards
FROM   Summer_Medals
WHERE  year = 2004
       AND medal = 'Gold'
       AND country IN ( 'DEN', 'NOR', 'SWE' )
GROUP  BY ROLLUP( country, gender )
ORDER  BY country ASC,
          gender ASC`

## Summarizing results
---

After ranking each country in the 2000 Olympics by gold medals awarded, you want to return the top 3 countries in one row, as a comma-separated string. In other words, turn this:

`| Country | Rank |
 |---------|------|
 | USA     | 1    |
 | RUS     | 2    |
 | AUS     | 3    |
 | ...     | ...  |`
 
into this:

`USA, RUS, AUS`

### Instructions

Rank countries by the medals they've been awarded.

In [13]:
%%sql

WITH Country_Medals 
     AS (SELECT country,
                COUNT(*) AS Medals
         FROM   summer_medals
         WHERE  year = 2000
                AND medal = 'Gold'
         GROUP  BY country)
SELECT country,
       RANK() OVER (ORDER BY Medals DESC) AS Rank
FROM   Country_Medals
ORDER  BY rank ASC 

 * sqlite:///data/summer.db
Done.


country,Rank
USA,1
RUS,2
AUS,3
CHN,4
GER,5
NED,6
ROU,6
HUN,8
CUB,9
FRA,9


Return the top 3 countries by medals awarded as one comma-separated string.

`WITH Country_Medals 
     AS (SELECT country,
                COUNT(*) AS Medals
         FROM   summer_medals
         WHERE  year = 2000
                AND medal = 'Gold'
         GROUP  BY country),
     Country_Ranks 
     AS (SELECT country,
                Rank() OVER (ORDER BY Medals DESC) AS Rank
         FROM   Country_Medals
         ORDER  BY rank ASC)
SELECT STRING_AGG(country, ', ')
FROM   Country_Medals
WHERE  rank <= 3 `